In [1]:
import numpy as np
import pandas as pd

from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from biopsykit.utils.dataframe_handling import multi_xs
import optuna
import pickle
import pingouin as pg


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from fau_colors import register_cmaps
register_cmaps()

sns.set_theme(context="talk", style="white", palette="faculties", font_scale=1.2)

%matplotlib widget

In [4]:
path = Path.cwd().parents[1].joinpath("../exports/baseline/")

# Test

In [5]:
pd.read_csv(path.joinpath("baseline_test_agg.csv"), index_col = ["Unnamed: 0", "Unnamed: 1"])

mean         std
always_wake  accuracy                   0.326152    0.116555
             precision                  0.000000    0.000000
             recall                     0.000000    0.000000
             f1                         0.000000    0.000000
             kappa                      0.000000    0.000000
             specificity                1.000000    0.000000
             mcc                        0.000000    0.000000
             sleep_onset                0.000000    0.000000
             wake_onset                 0.000000    0.000000
             total_sleep_duration       0.000000    0.000000
             net_sleep_duration         0.000000    0.000000
             bed_interval_start         0.000000    0.000000
             bed_interval_end           0.000000    0.000000
             sleep_efficiency           0.000000    0.000000
             sleep_onset_latency        0.000000    0.000000
             getup_latency              0.000000    0.000000
             wake_after_sleep_onset     0.000000    0.000000
always_sleep accuracy                   0.673848    0.116555
             precision                  0.673848    0.116555
             recall                     1.000000    0.000000
             f1                         0.799026    0.088893
             kappa                      0.000000    0.000000
             specificity                0.000000    0.000000
             mcc                        0.000000    0.000000
             sleep_onset               67.334821   70.022350
             wake_onset              1049.620536  148.279048
             total_sleep_duration     983.285714  152.957389
             net_sleep_duration       983.285714  152.957389
             bed_interval_start        67.334821   70.022350
             bed_interval_end        1049.620536  148.279048
             sleep_efficiency         100.000000    0.000000
             sleep_onset_latency        0.000000    0.000000
             getup_latency              0.000000    0.000000
             wake_after_sleep_onset     0.000000    0.000000
ground_truth accuracy                   1.000000    0.000000
             precision                  1.000000    0.000000
             recall                     1.000000    0.000000
             f1                         1.000000    0.000000
             kappa                      1.000000    0.000000
             specificity                1.000000    0.000000
             mcc                        1.000000    0.000000
             sleep_onset              133.955357  109.507940
             wake_onset              1043.312500  150.064622
             total_sleep_duration     983.285714  152.957389
             net_sleep_duration       736.946429  149.772374
             bed_interval_start        67.334821   70.022350
             bed_interval_end        1049.620536  148.279048
             sleep_efficiency          75.506580   12.950891
             sleep_onset_latency       66.620536   96.397690
             getup_latency              6.308036   25.120315
             wake_after_sleep_onset   161.638393  118.720414

# Train

In [6]:
pd.read_csv(path.joinpath("baseline_train_agg.csv"), index_col = ["Unnamed: 0", "Unnamed: 1"])

mean         std
always_wake  accuracy                   0.315766    0.115727
             precision                  0.000000    0.000000
             recall                     0.000000    0.000000
             f1                         0.000000    0.000000
             kappa                      0.000000    0.000000
             specificity                1.000000    0.000000
             mcc                        0.000000    0.000000
             sleep_onset                0.000000    0.000000
             wake_onset                 0.000000    0.000000
             total_sleep_duration       0.000000    0.000000
             net_sleep_duration         0.000000    0.000000
             bed_interval_start         0.000000    0.000000
             bed_interval_end           0.000000    0.000000
             sleep_efficiency           0.000000    0.000000
             sleep_onset_latency        0.000000    0.000000
             getup_latency              0.000000    0.000000
             wake_after_sleep_onset     0.000000    0.000000
always_sleep accuracy                   0.684234    0.115727
             precision                  0.684234    0.115727
             recall                     1.000000    0.000000
             f1                         0.806596    0.086769
             kappa                      0.000000    0.000000
             specificity                0.000000    0.000000
             mcc                        0.000000    0.000000
             sleep_onset               76.621652   71.762074
             wake_onset              1045.734375  159.863646
             total_sleep_duration     970.112723  160.533933
             net_sleep_duration       970.112723  160.533933
             bed_interval_start        76.621652   71.762074
             bed_interval_end        1045.734375  159.863646
             sleep_efficiency         100.000000    0.000000
             sleep_onset_latency        0.000000    0.000000
             getup_latency              0.000000    0.000000
             wake_after_sleep_onset     0.000000    0.000000
ground_truth accuracy                   1.000000    0.000000
             precision                  1.000000    0.000000
             recall                     1.000000    0.000000
             f1                         1.000000    0.000000
             kappa                      1.000000    0.000000
             specificity                1.000000    0.000000
             mcc                        1.000000    0.000000
             sleep_onset              136.338170  102.060025
             wake_onset              1040.293527  160.169414
             total_sleep_duration     970.112723  160.533933
             net_sleep_duration       741.742188  148.386147
             bed_interval_start        76.621652   71.762074
             bed_interval_end        1045.734375  159.863646
             sleep_efficiency          76.940696   12.112746
             sleep_onset_latency       59.716518   85.679942
             getup_latency              5.440848   22.454345
             wake_after_sleep_onset   147.549107  106.349155

# Full

In [7]:
df = pd.read_csv(path.joinpath("baseline_full.csv"), index_col=0, header=[0])

In [8]:
df.index.name = "metric"
df.columns.name = "subject"

In [9]:
df = pd.DataFrame(df.stack())

In [11]:
with open('../test_idx.pkl', 'rb') as f:
    test_idx_list = pickle.load(f)

In [12]:
#df = df.rename(columns={0:"sdf"})
df = df.unstack("metric")[0]
df

metric,accuracy,precision,recall,f1,kappa,specificity,mcc,confusion_matrix,sleep_onset,wake_onset,total_sleep_duration,net_sleep_duration,bed_interval_start,bed_interval_end,sleep_efficiency,sleep_onset_latency,getup_latency,wake_after_sleep_onset
subject,,,,,,,,,,,,,,,,,,
5697,1.0,1.0,1.0,1.0,1.0,1.0,1.0,"NoAgg(array([[421, 0],\n [ 0, 874]]))",228,1228,1108,874,122,1229,78.88086642599278,106,1,83
0219,1.0,1.0,1.0,1.0,1.0,1.0,1.0,"NoAgg(array([[334, 0],\n [ 0, 465]]))",170,712,604,465,110,713,76.98675496688742,60,1,38
4508,1.0,1.0,1.0,1.0,1.0,1.0,1.0,"NoAgg(array([[121, 0],\n [ 0, 915]]))",0,1035,1036,915,0,1035,88.32046332046332,0,0,121
1552,1.0,1.0,1.0,1.0,1.0,1.0,1.0,"NoAgg(array([[681, 0],\n [ 0, 457]]))",379,870,750,457,122,871,60.93333333333333,257,1,22
6796,1.0,1.0,1.0,1.0,1.0,1.0,1.0,"NoAgg(array([[119, 0],\n [ 0, 916]]))",0,1011,1013,916,0,1012,90.42448173741361,0,1,96
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1957,1.0,1.0,1.0,1.0,1.0,1.0,1.0,"NoAgg(array([[383, 0],\n [ 0, 954]]))",272,1281,1162,954,121,1282,82.09982788296041,151,1,56
3303,1.0,1.0,1.0,1.0,1.0,1.0,1.0,"NoAgg(array([[344, 0],\n [ 0, 633]]))",213,954,834,633,122,955,75.89928057553958,91,1,28
4252,1.0,1.0,1.0,1.0,1.0,1.0,1.0,"NoAgg(array([[356, 0],\n [ 0, 937]]))",200,1274,1293,937,0,1292,72.46713070378964,200,18,138


In [13]:
df_test = df.filter(items=test_idx_list, axis=0)

In [14]:
df_train = df.drop(labels=test_idx_list, axis=0)

In [15]:
pg.mwu(df_train["total_sleep_duration"].astype(float), df_test["total_sleep_duration"].astype(float)).round(4)

,U-val,alternative,p-val,RBC,CLES
MWU,95965.5,two-sided,0.3111,0.0437,0.4781


In [16]:
pg.mwu(df_train["wake_after_sleep_onset"].astype(float), df_test["wake_after_sleep_onset"].astype(float)).round(4)

,U-val,alternative,p-val,RBC,CLES
MWU,94462.5,two-sided,0.1738,0.0587,0.4707


In [17]:
pg.mwu(df_train["sleep_efficiency"].astype(float), df_test["sleep_efficiency"].astype(float)).round(4)

,U-val,alternative,p-val,RBC,CLES
MWU,106667.0,two-sided,0.1448,-0.0629,0.5315
